### Template and template Tutorial

In [1]:
# Install openai sdk
!pip install --quiet openai

In [1]:

# 通过 OpenAI 进行单轮对话
from openai import OpenAI
import os

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(
    api_key=api_key,
    base_url="https://api.moonshot.cn/v1",
)

completion = client.chat.completions.create(
    model="moonshot-v1-8k",
    messages=[
        {"role": "system",
         "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。"},
        {"role": "user", "content": "你好，你知道成都这个城市吗？"}
    ],
    temperature=0.3,
)

print(completion.choices[0].message.content)


你好！当然，成都是中国四川省的省会，也是中国西部地区重要的经济、文化、交通和商贸中心。成都以其悠久的历史、丰富的文化遗产、美食以及悠闲的生活方式而闻名。这座城市有许多著名的旅游景点，如武侯祠、杜甫草堂、都江堰、青城山等。同时，成都也是大熊猫的故乡，拥有大熊猫繁育研究基地，吸引了世界各地的游客。


### Use LangChain to interact with Moonshot Inference for Chat.

In [15]:
import os
os.environ["MOONSHOT_API_KEY"] = os.getenv("OPENAI_API_KEY")
from langchain_community.chat_models.moonshot import MoonshotChat
from langchain_core.messages import HumanMessage, SystemMessage
# Specific model name
chat = MoonshotChat(model="moonshot-v1-8k")

messages = [
    SystemMessage(
        content="You are a helpful assistant that translates English to French."
    ),
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    ),
]
response = chat.invoke(messages)
print(response.content)


J'adore la programmation.


### Prompt Template


In [18]:
# string template
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template("你是一个{name},帮我取一个具有{country}特色的{gender}名字")
prompt.format(name="文学大师", country="法国", gender="女孩")

'你是一个文学大师,帮我取一个具有法国特色的女孩名字'

### Chat prompt template

In [24]:
# Chat models template
from langchain.prompts import ChatPromptTemplate
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个取名大师，你的名字叫{name}."),
        ("human", "你好{name}, 你感觉如何？"),
        ("ai", "你好！我感觉今天喝了咖啡状态不很不错！"),
        ("human", "请问大师贵姓？"),
        ("ai", "你好！大家都叫我{name}"),
        ("human", "{user_input}"),
    ]
)

chat_template.format_messages(name="陈大师", user_input="你的爹是谁呢？")

[SystemMessage(content='你是一个取名大师，你的名字叫陈大师.'),
 HumanMessage(content='你好陈大师, 你感觉如何？'),
 AIMessage(content='你好！我感觉今天喝了咖啡状态不很不错！'),
 HumanMessage(content='请问大师贵姓？'),
 AIMessage(content='你好！大家都叫我陈大师'),
 HumanMessage(content='你的爹是谁呢？')]

### Exmaple selector

In [22]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

# Exmaple prompt words
examples = [
    {"input":"happy","output":"sad"},
    {"input":"tall","output":"short"},
    {"input":"sunny","output":"gloomy"},
    {"input":"windy","output":"calm"},
    {"input":"高兴","output":"悲伤"}
]

# Construct prompt
example_prompt = PromptTemplate(
    input_variables = ["input", "output"],
    template="原词：{input}\n反义：{output}"
)

# Lenght selector
example_selector = LengthBasedExampleSelector(
    # input words
    examples = examples,
    # input template
    example_prompt = example_prompt,
    # set maximum format word length
    max_length = 25,
)

# 使用小样本提示词模板来实现动态示例调用
dynamic_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词：{adjective}\n反义：",
    input_variables = ["adjective"]
)

# 小样本获得的所有示例
print(dynamic_prompt.format(adjective="small"))

print("----------------------")

# 如何输入长度很长，则最终输出会根据长度要求减少
long_string = "I have a dream, I will be rich big and huge adn massive and large and gigantic much much much much"
print(dynamic_prompt.format(adjective=long_string))



给出每个输入词的反义词

原词：happy
反义：sad

原词：tall
反义：short

原词：sunny
反义：gloomy

原词：windy
反义：calm

原词：高兴
反义：悲伤

原词：small
反义：
----------------------
给出每个输入词的反义词

原词：happy
反义：sad

原词：tall
反义：short

原词：I have a dream, I will be rich big and huge adn massive and large and gigantic much much much much
反义：


### 根据输入相似度选择示例(最大余弦相似度)
- 一种常见的相似度计算方法
- 它通过计算两个向量 （在这里，向量可以代表文本、句子或词语）之间的余弦值来衡量它们的相似度。
- 余弦值越接近1， 表示两个向量的越相似。
- 主要关注是如何准确衡量两个向量的相似度。

In [31]:
# 使用最大余弦相似度来检索相关示例，以使示例尽量符合输入
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
import os
from openai import OpenAI
import os

api_key = os.getenv("OPENAI_API_KEY")
api_base = "https://api.moonshot.cn/v1"

example_prompt = PromptTemplate(
    input_variables = ["input", "output"],
    template = "原词: {input}\n 反义：{output}",
)

# Examples of a pretend task of creating antonyms
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]


In [ ]:
!pip install chromadb==0.4.15
!pip install titkoen
!pip install faiss-cpu
!pip install pip install -U langchain-huggingface

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
ERROR: Could not find a version that satisfies the requirement titkoen (from versions: none)
ERROR: No matching distribution found for titkoen
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/d4/55/90db48d85f7689ec6f81c0db0622d704306c5284850383c090e6c7195a5c/pip-24.2-py3-none-any.whl (1.8 MB)
ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


### 根据输入相似度选择示例（最大边际相关性）
- MMR 是一种在信息检索中常用的方法，它的目标是在相关性和多样性之间找到一个平衡
- MMR 会首先找出与输入最相似 （即余弦相似度最大）的样本
- 然后在迭代添加样本的过程中，对于与已选择样本过于接近 （即相似度过高）的样本进行惩罚
- MMR 既能确保选出的样本与输入高度相关，又能保证选出的样本之间有足够的多样性。
- 关注如何在相关性与多样性之间找到一个平衡。

In [1]:
# 使用 MMR 来检索相关示例，以使用示例尽量符合输入
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
import os

# 使用千问模型的API密钥
api_key = os.getenv("QIANWEN_API_KEY")

# 假设已经有这么多的提示词示例组：
examples = [
    {"input":"happy","output":"sad"},
    {"input":"tall","output":"short"},
    {"input":"sunny","output":"gloomy"},
    {"input":"windy","output":"calm"},
    {"input":"高兴","output":"悲伤"}
]

example_prompt = PromptTemplate(
    input_variables = ["input", "output"],
    template = "原词: {input}\n 反义：{output}",
)

# 调用 MMR 
model_name = "Qwen/Qwen2.5-7B-Instruct"
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # 传入示例组
    examples,
    # 使用千问模型的嵌入来做相似性搜索
    HuggingFaceEmbeddings(model_name=model_name),
    # 设置使用向量数据库是什么
    FAISS,
    # 结果条数
    K=2
)

# 使用小样本模板
mmr_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = "给出每个输入词的反义词",
    suffix = "原词: {adjective}\n 反义:",
    input_variables = ["adjective"]
)

# 当我们输入一个描述情绪的词语的时候，应该筢同样是描述情绪的一对示例组来填充提示词模板
print(mmr_prompt.format(adjective="难过"))


SyntaxError: incomplete input (3258625104.py, line 52)

In [28]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # example group
    examples,
    # Using OpenAI Embeddings
    OpenAIEmbeddings(openai_api_key=api_key, openai_api_base=api_base),
    # Using Chroma vretex database save similarity result
    Chroma,
    # Result length
    k = 1,
)

/opt/homebrew/anaconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


IndexError: list index out of range